<a href="https://colab.research.google.com/github/NicoleRichards1998/FinRL/blob/master/JSE_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Install FinRL

In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-ej4xobpi
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-ej4xobpi
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-c0byrn7f/pyfolio_1a3a923b70d0496d8037724906fb7c2a
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-c0byrn7f/pyfolio_1a3a923b70d0496d8037724906fb7c2a
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-c0byrn7f/elegantrl_ea2660c2206a4db2b549689490169f82
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-c0byrn7f/elegantrl_ea2660c2206a4db2b549689490169f82
     |████████████████████████████████| 234 kB 52.2 MB/s 
     |████████████████████████████████| 177 kB 58.7 MB/s 
     |████████████████████████████████| 52.7 MB 148 kB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████

## Import related modules

In [1]:
from finrl.train import train
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib

import numpy as np
import pandas as pd
from pandas.core.arrays import string_

from datetime import datetime

from google.colab import files

import pytz
import exchange_calendars as tc

from stockstats import StockDataFrame as Sdf

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


## Import Dow Jones 30 Symbols

In [2]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)

In [3]:
JSEIndexes = [ 
  'SNH',
  'WHL',
 'FSR',
 'RMH',
 'BLU',
 'SSW',
 'LAB',
 'TGA',
 'SPG',
 'PPC',
 'OMU',
 'GFI',
 'NPK',
 'HAR',
 'LHC',
 'TSG',
 'ACL',
 'PPH',
 'SBK',
 'PPE',
 'CML',
 'MRF',
 'MTN',
 'AVI',
 'SOL',
 'EXX',
 'TRU',
 'NED',
 'TCP',
 'WSL'
 ]

In [4]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

In [5]:
tech_indicator_list = INDICATORS

In [7]:
url = 'https://raw.githubusercontent.com/NicoleRichards1998/data/main/unordered_JSE_Data.csv?token=GHSAT0AAAAAABRFXB6VIBIQZXAVCEBAGNV6YSUFO5A'
allDf = pd.read_csv(url)

### Step 3. Data Cleaning & Feature Engineering

In [6]:
trading_days = pd.date_range(start = '02/28/22', end = '03/25/22')

trading_days_List = trading_days[0:].strftime('%m/%d/%y').to_numpy().tolist()
count = 0
delete_number = 0
while count < 26:
    if (trading_days[count].weekday()>=5):
      del trading_days_List[delete_number]
      delete_number = delete_number - 1
    count = count + 1
    delete_number = delete_number + 1 

In [7]:
print(trading_days_List)

['02/28/22', '03/01/22', '03/02/22', '03/03/22', '03/04/22', '03/07/22', '03/08/22', '03/09/22', '03/10/22', '03/11/22', '03/14/22', '03/15/22', '03/16/22', '03/17/22', '03/18/22', '03/21/22', '03/22/22', '03/23/22', '03/24/22', '03/25/22']


## Download csv

In [218]:
url = 'https://raw.githubusercontent.com/NicoleRichards1998/data/main/CML.xlsx%20-%20Sheet1.csv?token=GHSAT0AAAAAABRFXB6VGC26MP7UGOKESVMYYSVK6KA'
df = pd.read_csv(url)
tic = 'CML'
df.head(7)

,Start,28/02/2022,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,End,28/03/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,BarTp=T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,CML SJ Index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,OPEN,NaN,CLOSE,NaN,HIGH,NaN,LOW,NaN,VOLUME
5,#NAME?,4690,#NAME?,4670,#NAME?,4690,#NAME?,4670,#NAME?,2132
6,2/28/22 9:05,4678,2/28/22 9:05,4678,2/28/22 9:05,4678,2/28/22 9:05,4678,2/28/22 9:05,725


## Delete bad stuff

In [219]:
df = df.iloc[4:, ]
df.columns = df.iloc[0]
df = df.iloc[1:, ]
df.columns = ['time', 'open','remove1', 'close','remove2', 'high','remove3', 'low','remove4', 'volume']
df.at[5,'time'] = '2/28/22 9:00'
df = df.drop(['remove1', 'remove2', 'remove3', 'remove4'], axis = 1)
df['tic'] = tic
df

,time,open,close,high,low,volume,tic
5,2/28/22 9:00,4690,4670,4690,4670,2132,CML
6,2/28/22 9:05,4678,4678,4678,4678,725,CML
7,2/28/22 9:06,4671,4670,4671,4670,4048,CML
8,2/28/22 9:14,4683,4705,4705,4683,744,CML
9,2/28/22 9:18,4675,4670,4675,4670,590,CML
...,...,...,...,...,...,...,...
4176,3/25/22 16:46,4474,4474,4474,4474,1235,CML
4177,3/25/22 16:47,4477,4479,4479,4477,1736,CML
4178,3/25/22 16:48,4477,4477,4477,4477,1011,CML
4179,3/25/22 16:49,4475,4478,4478,4475,672,CML


## Get clean

In [220]:
tic_list = np.unique(df.tic.values)
times = []

df = df.reset_index()
for i in range(df.shape[0]):
  try:
    df['time'][i] = pd.Timestamp(df['time'][i])
  except:
    df.drop(df.index[i])

for day in trading_days_List:
  current_time = pd.Timestamp(day + " 09:00")
  for i in range(481):
                times.append(current_time)
                current_time += pd.Timedelta(minutes=1)


for tic in tic_list:
            tmp_df = pd.DataFrame(
                columns=["open", "high", "low", "close", "volume"], index=times
            )
            tic_df = df[df.tic == tic]

            for i in range(tic_df.shape[0]):
                            tmp_df.loc[tic_df.iloc[i]['time']] = tic_df.iloc[i][
                                ["open", "high", "low", "close", "volume"]]
            
            if str(tmp_df.iloc[0]["close"]) == "nan":
                print('The price of the first row for ticker ', tic, ' is NaN. ', 
                      'It will filled with the first valid price.')
                for i in range(tmp_df.shape[0]):
                    if str(tmp_df.iloc[i]["close"]) != "nan":
                            first_valid_price = tmp_df.iloc[i]["close"]
                            tmp_df.iloc[0] = [first_valid_price,
                                      first_valid_price,
                                      first_valid_price,
                                      first_valid_price,
                                      0.0,]
                            break
              #if the close price of the first row is still NaN (All the prices are NaN in this case)
            if str(tmp_df.iloc[0]["close"]) == "nan":
                  print('Missing data for ticker: ', tic, ' . The prices are all NaN. Fill with 0.')
                  tmp_df.iloc[0] = [0.0, 0.0, 0.0, 0.0, 0.0,]
                        
              #forward filling row by row
            for i in range(tmp_df.shape[0]):
                  if str(tmp_df.iloc[i]["close"]) == "nan":
                      previous_close = tmp_df.iloc[i - 1]["close"]
                      if str(previous_close) == "nan":
                          raise ValueError
                      tmp_df.iloc[i] = [
                          previous_close,
                          previous_close,
                          previous_close,
                          previous_close,
                          0.0,
                      ]
            #tmp_df = tmp_df.astype(float)
            tmp_df["tic"] = tic
            tmp_df = tmp_df.iloc[0:9620, 0:6]
            new_df = tmp_df.reset_index()
            new_df = new_df.rename(columns={"index": "time"})

print(new_df)

                    time  open  high   low close  volume  tic
0    2022-02-28 09:00:00  4690  4690  4670  4670    2132  CML
1    2022-02-28 09:01:00  4670  4670  4670  4670     0.0  CML
2    2022-02-28 09:02:00  4670  4670  4670  4670     0.0  CML
3    2022-02-28 09:03:00  4670  4670  4670  4670     0.0  CML
4    2022-02-28 09:04:00  4670  4670  4670  4670     0.0  CML
...                  ...   ...   ...   ...   ...     ...  ...
9615 2022-03-25 16:56:00  4478  4478  4478  4478     0.0  CML
9616 2022-03-25 16:57:00  4478  4478  4478  4478     0.0  CML
9617 2022-03-25 16:58:00  4478  4478  4478  4478     0.0  CML
9618 2022-03-25 16:59:00  4478  4478  4478  4478     0.0  CML
9619 2022-03-25 17:00:00  4469  4469  4469  4469  299820  CML

[9620 rows x 7 columns]


## Save and download

In [221]:
cml = new_df

In [237]:
#all = all.append(new_df)
filename = tic + '_JSE_Minute_Data.csv'
cml.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [207]:
all

,time,open,high,low,close,volume,tic
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL
1,2022-02-28 09:01:00,845,845,845,845,6,ACL
2,2022-02-28 09:02:00,845,845,845,845,0.0,ACL
3,2022-02-28 09:03:00,845,845,845,845,0.0,ACL
4,2022-02-28 09:04:00,845,845,845,845,0.0,ACL
...,...,...,...,...,...,...,...
9615,2022-03-25 16:56:00,185,185,185,185,0.0,WSL
9616,2022-03-25 16:57:00,185,185,185,185,0.0,WSL
9617,2022-03-25 16:58:00,185,185,185,185,0.0,WSL
9618,2022-03-25 16:59:00,185,185,185,185,0.0,WSL


## Add indicators

In [8]:
url = 'https://raw.githubusercontent.com/NicoleRichards1998/data/main/JSE_Minute_Data_%2030_Tickers.csv?token=GHSAT0AAAAAABRFXB6VHVFZ2HEXH373ASYWYSWRVPA'
allDf = pd.read_csv(url)

In [19]:
df = allDf.copy()
df = df.reset_index()
for i in range(df.shape[0]):
  try:
    df['time'][i] = pd.Timestamp(df['time'][i])
  except:
    df.drop(df.index[i])
new_df = df.rename(columns={"time": "date"})
new_df

,index,date,open,high,low,close,volume,tic
0,0,2022-02-28 09:00:00,845,849,845,849,1395,ACL
1,1,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI
2,2,2022-02-28 09:00:00,539,539,539,539,1000,BLU
3,3,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML
4,4,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX
...,...,...,...,...,...,...,...,...
288595,288595,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA
288596,288596,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU
288597,288597,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG
288598,288598,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL


In [21]:
better = new_df.copy()
better = better.drop(['index'], axis = 1)
better

,date,open,high,low,close,volume,tic
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL
1,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI
2,2022-02-28 09:00:00,539,539,539,539,1000,BLU
3,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML
4,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX
...,...,...,...,...,...,...,...
288595,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA
288596,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU
288597,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG
288598,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL


In [22]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=True,
                    user_defined_feature = False)
df = fe.preprocess_data(better)
df

ValueError: ignored

In [15]:
df

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2/28/22 10:00,865,868,865,868,4238,ACL,0.000000,873.742641,865.257359,100.000000,66.666667,100.000000,868.000000,868.000000,0.000000
1,2/28/22 10:00,7225,7225,7225,7225,27,AVI,0.000000,873.742641,865.257359,100.000000,66.666667,100.000000,7225.000000,7225.000000,0.000000
2,2/28/22 10:00,540,540,540,540,0,BLU,0.000000,873.742641,865.257359,100.000000,66.666667,100.000000,540.000000,540.000000,0.000000
3,2/28/22 10:00,4660,4660,4660,4660,25,CML,0.000000,873.742641,865.257359,100.000000,66.666667,100.000000,4660.000000,4660.000000,0.000000
4,2/28/22 10:00,19472,19481,19471,19481,981,EXX,0.000000,873.742641,865.257359,100.000000,66.666667,100.000000,19481.000000,19481.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288595,3/9/22 9:59,16842,16843,16795,16795,406,TGA,-48.699516,17173.127671,16791.472329,45.532729,-154.480578,7.564084,17031.733333,17055.916667,26.477311
288596,3/9/22 9:59,5408,5417,5408,5417,407,TRU,8.335130,5453.023249,5379.176751,51.953781,38.030389,5.256646,5400.366667,5374.116667,26.477311
288597,3/9/22 9:59,1046,1046,1046,1046,0,TSG,0.598141,1046.000000,1046.000000,47.966128,40.000000,29.664825,1044.000000,1045.066667,26.477311
288598,3/9/22 9:59,5470,5470,5457,5470,4687,WHL,9.975150,5481.546780,5384.953220,53.875055,157.897277,32.434123,5429.700000,5435.533333,26.477311


In [13]:
with_indicators = df.copy()

In [198]:
API_KEY = "PKJSSCSTQJZ87LKDF5OP"
API_SECRET = "nJOIVicXAh8HZy958ZYcGOWqO4behHpIGJEHaHDL"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'
env = StockTradingEnv

In [199]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

Alpaca successfully connected


In [35]:
tech_indicator_list=[
            "macd",
            "boll_ub",
            "boll_lb",
            "rsi_30",
            "dx_30",
            "close_30_sma",
            "close_60_sma",
        ]

In [36]:
df = allDf.rename(columns={"time": "date"})
df = better.copy()
df = df.sort_values(by=["tic", "date"])
stock = Sdf.retype(df.copy())
unique_ticker = stock.tic.unique()

for indicator in tech_indicator_list:
            indicator_df = pd.DataFrame()
            for i in range(len(unique_ticker)):
                #print(unique_ticker[i], i)
                temp_indicator = stock[stock.tic == unique_ticker[i]][indicator]
                temp_indicator = pd.DataFrame(temp_indicator)
                temp_indicator["tic"] = unique_ticker[i]
                #print(len(df[df.tic == unique_ticker[i]]['date'].to_list()))
                temp_indicator["date"] = df[df.tic == unique_ticker[i]][
                    "date"
                ].to_list()
                indicator_df = indicator_df.append(temp_indicator, ignore_index=True)
            df['date'] = df['date'].astype('datetime64[ns]')
            indicator_df['date'] = indicator_df['date'].astype('datetime64[ns]')
            df = df.merge(
                indicator_df[["tic", "date", indicator]], on=["tic", "date"], how="left"
            )
df

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,NaN,NaN,NaN,NaN,849.000000,849.000000
1,2022-02-28 09:01:00,845,845,845,845,6,ACL,-0.089744,852.656854,841.343146,0.000000,NaN,847.000000,847.000000
2,2022-02-28 09:02:00,845,845,845,845,0,ACL,-0.114351,850.952135,841.714531,0.000000,NaN,846.333333,846.333333
3,2022-02-28 09:03:00,845,845,845,845,0,ACL,-0.122741,850.000000,842.000000,0.000000,NaN,846.000000,846.000000
4,2022-02-28 09:04:00,845,845,845,845,0,ACL,-0.124715,849.377709,842.222291,0.000000,NaN,845.800000,845.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288595,2022-03-25 16:56:00,185,185,185,185,0,WSL,-0.072443,185.000009,184.999991,0.000009,100.0,185.000000,185.533333
288596,2022-03-25 16:57:00,185,185,185,185,0,WSL,-0.067131,185.000009,184.999991,0.000009,100.0,185.000000,185.466667
288597,2022-03-25 16:58:00,185,185,185,185,0,WSL,-0.062204,185.000009,184.999991,0.000009,100.0,185.000000,185.400000
288598,2022-03-25 16:59:00,185,185,185,185,0,WSL,-0.057634,185.000009,184.999991,0.000009,100.0,185.000000,185.333333


In [38]:
def add_turbulence(data):
        """
        add turbulence index from a precalcualted dataframe
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        turbulence_index = calculate_turbulence(df)
        df = df.merge(turbulence_index, on="date")
        df = df.sort_values(["date", "tic"]).reset_index(drop=True)
        return df

def calculate_turbulence(data):
        """calculate turbulence index based on dow 30"""
        # can add other market assets
        df = data.copy()
        df_price_pivot = df.pivot(index="date", columns="tic", values="close")
        # use returns to calculate turbulence
        df_price_pivot = df_price_pivot.pct_change()

        unique_date = df.date.unique()
        # start after a year
        start = 252
        turbulence_index = [0] * start
        # turbulence_index = [0]
        count = 0
        for i in range(start, len(unique_date)):
            current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
            # use one year rolling window to calcualte covariance
            hist_price = df_price_pivot[
                (df_price_pivot.index < unique_date[i])
                & (df_price_pivot.index >= unique_date[i - 252])
            ]
            # Drop tickers which has number missing values more than the "oldest" ticker
            filtered_hist_price = hist_price.iloc[
                hist_price.isna().sum().min() :
            ].dropna(axis=1)

            cov_temp = filtered_hist_price.cov()
            current_temp = current_price[[x for x in filtered_hist_price]] - np.mean(
                filtered_hist_price, axis=0
            )
            # cov_temp = hist_price.cov()
            # current_temp=(current_price - np.mean(hist_price,axis=0))

            temp = current_temp.values.dot(np.linalg.pinv(cov_temp)).dot(
                current_temp.values.T
            )
            if temp > 0:
                count += 1
                if count > 2:
                    turbulence_temp = temp[0][0]
                else:
                    # avoid large outlier because of the calculation just begins
                    turbulence_temp = 0
            else:
                turbulence_temp = 0
            turbulence_index.append(turbulence_temp)

        turbulence_index = pd.DataFrame(
            {"date": df_price_pivot.index, "turbulence": turbulence_index}
        )
        return turbulence_index

In [39]:
with_turb = add_turbulence(df)
with_turb

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,NaN,NaN,NaN,NaN,849.000000,849.000000,0.000000
1,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI,0.000000,NaN,NaN,NaN,NaN,7219.000000,7219.000000,0.000000
2,2022-02-28 09:00:00,539,539,539,539,1000,BLU,0.000000,NaN,NaN,NaN,NaN,539.000000,539.000000,0.000000
3,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML,0.000000,NaN,NaN,NaN,NaN,4670.000000,4670.000000,0.000000
4,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX,0.000000,NaN,NaN,NaN,NaN,18851.000000,18851.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288595,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA,-35.054682,17130.827940,16736.172060,54.207299,28.153561,16963.966667,17021.800000,408.808293
288596,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU,10.672713,5822.352803,5789.447197,63.936825,7.077874,5788.100000,5762.183333,408.808293
288597,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG,0.497467,1127.368915,1119.531085,46.866491,8.864078,1123.633333,1122.183333,408.808293
288598,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL,1.508432,5619.320849,5586.079151,60.145070,23.568954,5599.733333,5590.716667,408.808293


In [32]:
with_turb = with_turb.sort_values(by=["date", "tic"])
with_turb

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,close_30_sma,close_60_sma
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,NaN,NaN,849.000000,849.000000
9620,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI,0.000000,NaN,NaN,7219.000000,7219.000000
19240,2022-02-28 09:00:00,539,539,539,539,1000,BLU,0.000000,NaN,NaN,539.000000,539.000000
28860,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML,0.000000,NaN,NaN,4670.000000,4670.000000
38480,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX,0.000000,NaN,NaN,18851.000000,18851.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
250119,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA,-35.054682,17130.827940,16736.172060,16963.966667,17021.800000
259739,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU,10.672713,5822.352803,5789.447197,5788.100000,5762.183333
269359,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG,0.497467,1127.368915,1119.531085,1123.633333,1122.183333
278979,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL,1.508432,5619.320849,5586.079151,5599.733333,5590.716667


In [40]:
with_turb = with_turb.fillna(0)
with_turb

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,0.000000,0.000000,0.000000,0.000000,849.000000,849.000000,0.000000
1,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI,0.000000,0.000000,0.000000,0.000000,0.000000,7219.000000,7219.000000,0.000000
2,2022-02-28 09:00:00,539,539,539,539,1000,BLU,0.000000,0.000000,0.000000,0.000000,0.000000,539.000000,539.000000,0.000000
3,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML,0.000000,0.000000,0.000000,0.000000,0.000000,4670.000000,4670.000000,0.000000
4,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX,0.000000,0.000000,0.000000,0.000000,0.000000,18851.000000,18851.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288595,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA,-35.054682,17130.827940,16736.172060,54.207299,28.153561,16963.966667,17021.800000,408.808293
288596,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU,10.672713,5822.352803,5789.447197,63.936825,7.077874,5788.100000,5762.183333,408.808293
288597,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG,0.497467,1127.368915,1119.531085,46.866491,8.864078,1123.633333,1122.183333,408.808293
288598,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL,1.508432,5619.320849,5586.079151,60.145070,23.568954,5599.733333,5590.716667,408.808293


In [41]:
filename = 'JSE_Minute_Data_With_Turbulance_Indicator.csv'
with_turb.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Split into seperate days

In [287]:
dic = {}
count = 0
for day in trading_days_List:
  file_name = str(day)
  for i in range(len(with_indicators)):
    df = pd.DataFrame()
    if (with_indicators['date'][i]) != -1:
      alone = with_indicators.iloc[[i]]
      df = df.append(alone)
    else:
      count = count + 1
  dic[day] = df

dic



KeyboardInterrupt: ignored

In [289]:
with_indicators

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,close_30_sma,close_60_sma
0,2022-02-28 09:00:00,845,849,845,849,1395,ACL,0.000000,NaN,NaN,849.000000,849.000000
9620,2022-02-28 09:00:00,7219,7219,7219,7219,15,AVI,0.000000,NaN,NaN,7219.000000,7219.000000
19240,2022-02-28 09:00:00,539,539,539,539,1000,BLU,0.000000,NaN,NaN,539.000000,539.000000
28860,2022-02-28 09:00:00,4690,4690,4670,4670,2132,CML,0.000000,NaN,NaN,4670.000000,4670.000000
38480,2022-02-28 09:00:00,18851,18851,18851,18851,600,EXX,0.000000,NaN,NaN,18851.000000,18851.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
250119,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA,-35.054682,17130.827940,16736.172060,16963.966667,17021.800000
259739,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU,10.672713,5822.352803,5789.447197,5788.100000,5762.183333
269359,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG,0.497467,1127.368915,1119.531085,1123.633333,1122.183333
278979,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL,1.508432,5619.320849,5586.079151,5599.733333,5590.716667


In [47]:

#df = with_indicators.reset_index()
x = 14430
df = with_turb.copy()
df1 = df.iloc[0:x,]
df2 = df.iloc[(x):(2*x),]
df3 = df.iloc[(2*x):(3*x), ]
df4 = df.iloc[(3*x):(4*x),]
df5 = df.iloc[(4*x):(5*x),]
df6 = df.iloc[(5*x):(6*x),]
df7 = df.iloc[(6*x):(7*x),]
df8 = df.iloc[(7*x):(8*x), ]
df9 = df.iloc[(8*x):(9*x), ]
df10 = df.iloc[(9*x):(10*x), ]
df11 = df.iloc[(10*x):(11*x), ]
df12 = df.iloc[(11*x):(12*x), ]
df13 = df.iloc[(12*x):(13*x), ]
df14 = df.iloc[(13*x):(14*x), ]
df15 = df.iloc[(14*x):(15*x), ]
df16 = df.iloc[(15*x):(16*x), ]
df17 = df.iloc[(16*x):(17*x), ]
df18 = df.iloc[(17*x):(18*x), ]
df19 = df.iloc[(18*x):(19*x), ]
df20 = df.iloc[(19*x):(20*x), ]
df20

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
274170,2022-03-25 09:00:00,938,938,938,938,20,ACL,2.652796,942.141862,934.858138,64.573162,40.519000,934.766667,929.000000,5003.334005
274171,2022-03-25 09:00:00,6646,6646,6646,6646,99,AVI,-4.709030,6776.166901,6687.233099,21.253920,78.143925,6730.933333,6729.566667,5003.334005
274172,2022-03-25 09:00:00,576,576,576,576,0,BLU,-0.072203,583.609109,574.990891,35.224593,48.477952,579.133333,579.116667,5003.334005
274173,2022-03-25 09:00:00,4450,4450,4450,4450,9,CML,-1.077461,4462.441948,4451.158052,42.539006,28.076813,4458.800000,4458.216667,5003.334005
274174,2022-03-25 09:00:00,22000,22000,22000,22000,750,EXX,1.434256,21967.575047,21842.124953,60.903557,34.025235,21906.066667,21909.216667,5003.334005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288595,2022-03-25 17:00:00,17061,17061,17061,17061,210861,TGA,-35.054682,17130.827940,16736.172060,54.207299,28.153561,16963.966667,17021.800000,408.808293
288596,2022-03-25 17:00:00,5796,5796,5796,5796,306624,TRU,10.672713,5822.352803,5789.447197,63.936825,7.077874,5788.100000,5762.183333,408.808293
288597,2022-03-25 17:00:00,1122,1122,1122,1122,25156,TSG,0.497467,1127.368915,1119.531085,46.866491,8.864078,1123.633333,1122.183333,408.808293
288598,2022-03-25 17:00:00,5613,5613,5613,5613,581150,WHL,1.508432,5619.320849,5586.079151,60.145070,23.568954,5599.733333,5590.716667,408.808293


In [ ]:
dic = {}


### Step 4. Transform to numpy array

In [48]:
def df_to_array(df, tech_indicator_list, if_vix):
        df = df.copy()
        unique_ticker = df.tic.unique()
        if_first_time = True
        for tic in unique_ticker:
            if if_first_time:
                price_array = df[df.tic == tic][["close"]].values
                tech_array = df[df.tic == tic][tech_indicator_list].values
                if if_vix:
                    turbulence_array = df[df.tic == tic]["VIXY"].values
                else:
                    turbulence_array = df[df.tic == tic]["turbulence"].values
                if_first_time = False
            else:
                price_array = np.hstack(
                    [price_array, df[df.tic == tic][["close"]].values]
                )
                tech_array = np.hstack(
                    [tech_array, df[df.tic == tic][tech_indicator_list].values]
                )
        print("Successfully transformed into array")
        return price_array, tech_array, turbulence_array

In [50]:
price_array, tech_array, turbulence_array = df_to_array(df1, tech_indicator_list, if_vix=False)

Successfully transformed into array


In [51]:
price_array

array([[ 849, 7219,  539, ..., 1135, 4950,  166],
       [ 845, 7217,  539, ..., 1135, 4924,  166],
       [ 845, 7217,  539, ..., 1135, 4925,  166],
       ...,
       [ 885, 7166,  530, ..., 1158, 5049,  173],
       [ 885, 7166,  530, ..., 1158, 5049,  173],
       [ 879, 7165,  530, ..., 1164, 5050,  173]])

# Part 2: Train the agent

## Train

In [ ]:
def Train(df, model_name, rllib_params, filename):
  price_array, tech_array, turbulence_array = df_to_array(df, tech_indicator_list, if_vix=False)
        
  agent_rllib = DRLAgent_rllib(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
        )
  model, model_config = agent_rllib.get_model(model_name)
  model_config["lr"] = rllib_params["lr"]
  model_config["train_batch_size"] = rllib_params["train_batch_size"]
  model_config["gamma"] = rllib_params["gamma"]
        # ray.shutdown()
  trained_model = agent_rllib.train_model(
            model=model,
            model_name=model_name,
            model_config=model_config,
            total_episodes=480,
        )
  trained_model.save(filename)

In [ ]:
Train(df1, 'ppo', )

In [ ]:
ERL_PARAMS = {"learning_rate": 3e-6,"batch_size": 2048,"gamma":  0.985,
        "seed":312,"net_dimension":512, "target_step":5000, "eval_gap":30,
        "eval_times":1} 
#if you want to use larger datasets (change to longer period), and it raises error, 
#please try to increase "target_step". It should be larger than the episode steps. 

In [ ]:
train(
      model_name='ppo', 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl', #current_working_dir
      break_step=1e5)

Alpaca successfully connected
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data clean finished!
Succesfully add technical indicators
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data clean finished!
Successfully transformed into array
| Remove cwd: ./papertrading_erl
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  3.90e+03    1.01 |
0  3.90e+03    1.01 |    

## Test

In [ ]:
account_value_erl=test(start_date = '2021-10-18', 
                      end_date = '2021-10-19',
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= '1Min', 
                      technical_indicator_list= INDICATORS,
                      drl_lib='elegantrl', 
                      env=env, 
                      model_name='ppo', 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd='./papertrading_erl',
                      net_dimension = 512)

Alpaca successfully connected
Data before 2021-10-18T15:59:00-04:00 is successfully fetched
Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Succesfully add technical indicators
Data before 2021-10-18T15:59:00-04:00 is successfully fetched
Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Successfully transformed into array
price_array:  780
Test Finished!
episode_return 1.0075374272908477


## Use full data to train 

After tuning well, retrain on the training and testing sets

In [ ]:
train(start_date = '2021-10-11', 
      end_date = '2021-10-19',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env, 
      model_name='ppo', 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl_retrain',
      break_step=5e4)

Alpaca successfully connected
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data before 2021-10-16T15:59:00-04:00 is successfully fetched
Data before 2021-10-17T15:59:00-04:00 is successfully fetched


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


Data before 2021-10-18T15:59:00-04:00 is successfully fetched


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Succesfully add technical indicators
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data before 2021-10-16T15:59:00-04:00 is successfully fetched
Data before 2021-10-17T15:59:00-04:00 is successfully fetched
Data before 2021-10-18T15:59:00-04:00 is successfully fetched
Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Successfully transformed into array
| Remove cwd: ./papertrading_erl_retrain
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  2.73e+03    1.01 |
0  2.73e+03    1.01 |    1.01    0.0   2728    

# Part 3: Deploy the agent

## Setup Alpaca Paper trading environment

In [ ]:
import datetime
import threading
from finrl.finrl_meta.data_processors.processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=30, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                from elegantrl.agents.agent import AgentPPO
                from elegantrl.train.run import init_agent
                from elegantrl.train.config import Arguments
                #load agent
                config = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                args = Arguments(agent=AgentPPO, env=StockEnvEmpty(config))
                args.cwd = cwd
                args.net_dim = net_dim
                # load agent
                try:
                    agent = init_agent(args, gpu_id = 0)
                    self.act = agent.act
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                        
            elif drl_lib == 'rllib':
                from ray.rllib.agents import ppo
                from ray.rllib.agents.ppo.ppo import PPOTrainer
                
                config = ppo.DEFAULT_CONFIG.copy()
                config['env'] = StockEnvEmpty
                config["log_level"] = "WARN"
                config['env_config'] = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                trainer = PPOTrainer(env=StockEnvEmpty, config=config)
                trainer.restore(cwd)
                try:
                    trainer.restore(cwd)
                    self.agent = trainer
                    print("Restoring from checkpoint path", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            elif drl_lib == 'stable_baselines3':
                from stable_baselines3 import PPO
                
                try:
                    #load agent
                    self.model = PPO.load(cwd)
                    print("Successfully load model", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.asset_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break
            
            '''# Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")
    
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              tSubmitOrder.join()
    
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)'''
            
          else:
            trade = threading.Thread(target=self.trade)
            trade.start()
            trade.join()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
                
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
                
        else:  # sell all when turbulence
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval='1Min',
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh
    
class StockEnvEmpty(gym.Env):
    #Empty Env used for loading rllib agent
    def __init__(self,config):
      state_dim = config['state_dim']
      action_dim = config['action_dim']
      self.env_num = 1
      self.max_step = 10000
      self.env_name = 'StockEnvEmpty'
      self.state_dim = state_dim  
      self.action_dim = action_dim
      self.if_discrete = False  
      self.target_return = 9999
      self.observation_space = gym.spaces.Box(low=-3000, high=3000, shape=(state_dim,), dtype=np.float32)
      self.action_space = gym.spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        
    def reset(self):
        return 

    def step(self, actions):
        return

## Run Paper trading

In [ ]:
print(DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [ ]:
state_dim

333

In [ ]:
action_dim

30

In [ ]:
paper_trading_erl = AlpacaPaperTrading(ticker_list = DOW_30_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './papertrading_erl_retrain', 
                                       net_dim = 512, 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=1e2)
paper_trading_erl.run()

Waiting for market to open...
1219 minutes til market open.
1218 minutes til market open.


KeyboardInterrupt: ignored

# Part 4: Check Portfolio Performance

In [ ]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from finrl.plot import backtest_stats
import matplotlib.pyplot as plt

In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
                                pd.Timestamp(end,tz=pytz.UTC))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def DIA_history(start):
    data_df = yf.download(['^DJI'],start=start, interval="5m")
    data_df = data_df.iloc[48:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [ ]:
df_erl, cumu_erl = alpaca_history(key=API_KEY, 
                                  secret=API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2021-10-19', #must be within 1 month
                                  end='2021-10-22') #change the date if error occurs


In [ ]:
df_djia, cumu_djia = DIA_history(start='2021-10-19')


[*********************100%***********************]  1 of 1 completed


In [ ]:
print(df_erl)


                           profit_loss  profit_loss_pct      equity
timestamp                                                          
2021-10-19 09:30:00-04:00     -9324.06        -0.009005  1026133.65
2021-10-19 09:35:00-04:00    -10106.67        -0.009761  1025351.04
2021-10-19 09:40:00-04:00     -8463.56        -0.008174  1026994.15
2021-10-19 09:45:00-04:00     -9907.60        -0.009568  1025550.11
2021-10-19 09:50:00-04:00     -8516.10        -0.008224  1026941.61
...                                ...              ...         ...
2021-10-22 15:35:00-04:00      1560.63         0.001507  1037018.34
2021-10-22 15:40:00-04:00       729.00         0.000704  1036186.71
2021-10-22 15:45:00-04:00       546.57         0.000528  1036004.28
2021-10-22 15:50:00-04:00       459.65         0.000444  1035917.36
2021-10-22 15:55:00-04:00       862.31         0.000833  1036320.02

[312 rows x 3 columns]


In [ ]:
print(df_djia)

                                   Open          High  ...     Adj Close    Volume
Datetime                                               ...                        
2021-10-19 13:30:00-04:00  35428.171875  35434.890625  ...  35434.699219   2301420
2021-10-19 13:35:00-04:00  35435.179688  35440.160156  ...  35437.269531   1806707
2021-10-19 13:40:00-04:00  35437.480469  35437.710938  ...  35430.851562   1797519
2021-10-19 13:45:00-04:00  35430.601562  35432.808594  ...  35423.058594   1917764
2021-10-19 13:50:00-04:00  35422.929688  35435.828125  ...  35435.691406   2018081
...                                 ...           ...  ...           ...       ...
2021-10-22 15:35:00-04:00  35708.800781  35710.929688  ...  35684.738281   3547180
2021-10-22 15:40:00-04:00  35684.339844  35692.960938  ...  35675.769531   4243899
2021-10-22 15:45:00-04:00  35675.710938  35680.800781  ...  35670.238281   5178920
2021-10-22 15:50:00-04:00  35670.808594  35699.308594  ...  35692.921875   9231104
2021

In [ ]:
df_erl.tail()

,profit_loss,profit_loss_pct,equity
timestamp,,,
2021-10-22 15:35:00-04:00,1560.63,0.001507,1037018.34
2021-10-22 15:40:00-04:00,729.00,0.000704,1036186.71
2021-10-22 15:45:00-04:00,546.57,0.000528,1036004.28
2021-10-22 15:50:00-04:00,459.65,0.000444,1035917.36
2021-10-22 15:55:00-04:00,862.31,0.000833,1036320.02


In [ ]:
returns_erl = cumu_erl -1 
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of dia return: ', returns_dia.shape[0])

len of erl return:  312
len of dia return:  264


## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_dia, label = 'DJIA', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2021', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')